<a href="https://colab.research.google.com/github/Jarvis-BITS/midodrine-mimic-iv/blob/main/Midodrine_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from google.cloud import bigquery

run if TPU's are *connected*

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

run if GPU's connected

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Authenticate with GCP account having MIMIC-IV dataset

In [3]:
from google.colab import auth

auth.authenticate_user()
print('AuTHENticated')
 

AuTHENticated


In [4]:
%load_ext google.colab.data_table

In [5]:
%load_ext google.cloud.bigquery

In [6]:
project_id = 'mit-midodrine' #Enter your project ID from GCP bigquey here
bqclient = bigquery.Client(project=project_id)

## BigQuery Data Extraction

In [7]:
population = """
WITH patient_details AS (
  SELECT
  ie.subject_id, ie.hadm_id, ie.stay_id
  , pat.gender, pat.dod
  , adm.admittime, adm.dischtime
  , DATETIME_DIFF(adm.dischtime, adm.admittime, DAY) AS los_hospital
  /*	
  , pa.anchor_age
	, pa.anchor_year
  */
  , DATETIME_DIFF(adm.admittime, DATETIME(pat.anchor_year, 1, 1, 0, 0, 0), YEAR) + pat.anchor_age AS age
  , adm.ethnicity
  , adm.hospital_expire_flag
  /*  
  , DENSE_RANK() OVER (PARTITION BY adm.subject_id ORDER BY adm.admittime) AS hospstay_seq
  , CASE
  WHEN DENSE_RANK() OVER (PARTITION BY adm.subject_id ORDER BY adm.admittime) = 1 THEN True
  ELSE False END AS first_hosp_stay
  */
-- icu level factors
  , ie.intime AS icu_intime, ie.outtime AS icu_outtime
  , ROUND(DATETIME_DIFF(ie.outtime, ie.intime, HOUR)/24.0, 2) AS los_icu
  , RANK() OVER (PARTITION BY adm.subject_id ORDER BY ie.intime) AS icu_order
  /*  
  , DENSE_RANK() OVER (PARTITION BY ie.hadm_id ORDER BY ie.intime) AS icustay_seq 
-- first ICU stay *for the current hospitalization*
  , CASE
  WHEN DENSE_RANK() OVER (PARTITION BY ie.hadm_id ORDER BY ie.intime) = 1 THEN True
  ELSE False END AS first_icu_stay
  */
  FROM `physionet-data.mimic_icu.icustays` ie
  INNER JOIN `physionet-data.mimic_core.admissions` adm
  ON ie.hadm_id = adm.hadm_id
  INNER JOIN `physionet-data.mimic_core.patients` pat
  ON ie.subject_id = pat.subject_id
  )
  , midodrine AS (
    SELECT hadm_id, starttime AS drug_starttime
    FROM `physionet-data.mimic_hosp.pharmacy`
    WHERE lower(medication) like 'midodrine'
  )
  , midodrine_exclude AS (
    SELECT stay_id,
    CASE WHEN LOGICAL_AND(drug_starttime < DATE_TRUNC(intime - interval '1' day, DAY) OR drug_starttime > outtime) THEN 1 
    ELSE 0 END AS midodrine_exclude
    FROM `physionet-data.mimic_icu.icustays` 
    LEFT JOIN midodrine USING (hadm_id)
    GROUP BY stay_id
  )
  , midodrine_include AS (
    SELECT stay_id,
    CASE WHEN LOGICAL_AND(drug_starttime is NULL)
    OR LOGICAL_OR(drug_starttime BETWEEN DATE_TRUNC(intime - interval '1' day, DAY) AND outtime) THEN 1
    ELSE 0 END AS midodrine_include
    FROM `physionet-data.mimic_icu.icustays`  
    LEFT JOIN midodrine USING (hadm_id)
    GROUP BY stay_id
  )
  , midodrine_first AS (
  SELECT stay_id, min(drug_starttime) AS drug_starttime 
  FROM `physionet-data.mimic_icu.icustays` LEFT JOIN midodrine mi USING (hadm_id)
  WHERE drug_starttime BETWEEN DATE_TRUNC(intime - interval '1' day, DAY) AND outtime
  GROUP BY stay_id
  )
 , midodrine_dose AS (
  SELECT ic.stay_id, AVG(doses_per_24_hrs) AS avg_doses_per_24_hrs
  FROM `physionet-data.mimic_hosp.pharmacy` ph
  INNER JOIN `physionet-data.mimic_icu.icustays`ic
  ON ph.hadm_id = ic.hadm_id
  GROUP BY ic.stay_id
  )
  , population AS (
    SELECT * 
    FROM (SELECT DISTINCT stay_id, first_careunit, intime, outtime FROM `physionet-data.mimic_icu.icustays`) a
    LEFT JOIN patient_details USING (stay_id)
    LEFT JOIN midodrine_first USING (stay_id)
    LEFT JOIN midodrine_exclude USING (stay_id)
    LEFT JOIN midodrine_include USING (stay_id)
    LEFT JOIN midodrine_dose USING (stay_id)
    )
    SELECT * FROM population
"""

In [8]:
cohort_string = """
, cohort AS(
  SELECT *
    ,CASE WHEN drug_starttime is null THEN 1 ELSE 0 END AS drug
  FROM population
  WHERE age >= 18
    AND icu_order = 1
    AND (first_careunit LIKE '%MICU%' OR first_careunit LIKE '%SICU%')
    AND midodrine_include = 1
)
SELECT * FROM cohort
"""

In [9]:
comorbidities_string = """
WITH charlson AS(
  SELECT subject_id, hadm_id
  , renal_diseASe AS Renal
  , severe_liver_diseASe AS severe_liver
  , mild_liver_diseASe AS mild_liver
  , chronic_pulmonary_diseASe AS CPD_or_COPD
  , malignant_cancer AS Maligancy
  FROM `physionet-data.mimic_derived.charlson` char
)
, diag AS
(
  SELECT 
      hadm_id
      , CASE WHEN icd_version = 9 THEN icd_code ELSE NULL END AS icd9_code
      , CASE WHEN icd_version = 10 THEN icd_code ELSE NULL END AS icd10_code
  FROM `physionet-data.mimic_hosp.diagnoses_icd` diag
)
, addition AS(
  SELECT ad.hadm_id
  -- CAD
  , MAX(CASE WHEN
      icd9_code LIKE '414%'
      OR
      SUBSTR(icd10_code, 1, 4) = 'I251'
      THEN 1 
      ELSE 0 END) AS CAD
  -- AFIB
  , MAX(CASE WHEN
      icd9_code LIKE '4273%'
      OR
      SUBSTR(icd10_code, 1, 3) = 'I48'
      THEN 1 
      ELSE 0 END) AS AFIB
  FROM `physionet-data.mimic_core.admissions` ad
  LEFT JOIN diag
  ON ad.hadm_id = diag.hadm_id
  GROUP BY ad.hadm_id
)
, comorbidities AS(
  SELECT ic.stay_id
    , AVG(charlson.CPD_or_COPD) AS CPD_or_COPD_rate
    , AVG(charlson.Maligancy) AS Maligancy_rate
    , AVG(charlson.mild_liver) AS mild_liver_rate
    , AVG(charlson.severe_liver) AS severe_liver_rate
    , AVG(charlson.Renal) AS Renal_rate
    , AVG(addition.CAD) AS CAD_rate
    , AVG(addition.AFIB) AS AFIB_rate
  FROM `physionet-data.mimic_icu.icustays` ic
  LEFT JOIN charlson USING (hadm_id)
  LEFT JOIN addition USING (hadm_id)
  GROUP BY ic.stay_id
)
SELECT * FROM comorbidities
"""

In [10]:
weight_string = """
-- This query extracts weights for adult ICU patients WITH start/stop times
-- if an admission weight is given, THEN this is ASsigned from intime to outtime
WITH wt_stg AS
(
    SELECT
        c.stay_id
      , c.charttime
      , CASE WHEN c.itemid = 226512 THEN 'admit'
          ELSE 'daily' end AS weight_type
      -- TODO: eliminate obvious outliers if there is a reasonable weight
      , c.valuenum AS weight
    FROM `physionet-data.mimic_icu.chartevents` c
    WHERE c.valuenum IS NOT NULL
      AND c.itemid in
      (
          226512 -- Admit Wt
          , 224639 -- Daily Weight
      )
      AND c.valuenum > 0
)
-- ASsign AScending row number
, wt_stg1 AS
(
  SELECT
      stay_id
    , charttime
    , weight_type
    , weight
    , ROW_NUMBER() OVER (partition by stay_id, weight_type order by charttime) AS rn
  from wt_stg
  WHERE weight IS NOT NULL
)
-- change charttime to intime for the first admission weight recorded
, wt_stg2 AS
(
  SELECT 
      wt_stg1.stay_id
    , ie.intime, ie.outtime
    , wt_stg1.weight_type
    , CASE WHEN wt_stg1.weight_type = 'admit' and wt_stg1.rn = 1
        THEN DATETIME_SUB(ie.intime, INTERVAL '2' HOUR)
      ELSE wt_stg1.charttime end AS starttime
    , wt_stg1.weight
  from wt_stg1
  INNER JOIN `physionet-data.mimic_icu.icustays` ie
    on ie.stay_id = wt_stg1.stay_id
)
, wt_stg3 AS
(
  SELECT
    stay_id
    , intime, outtime
    , starttime
    , coalesce(
        LEAD(starttime) OVER (PARTITION BY stay_id ORDER BY starttime),
        DATETIME_ADD(outtime, INTERVAL '2' HOUR)
      ) AS endtime
    , weight
    , weight_type
  from wt_stg2
)
-- this table is the start/stop times from admit/daily weight in charted data
, wt1 AS
(
  SELECT
      stay_id
    , starttime
    , coalesce(endtime,
      LEAD(starttime) OVER (partition by stay_id order by starttime),
      -- impute ICU discharge AS the end of the final weight meASurement
      -- plus a 2 hour "fuzziness" window
      DATETIME_ADD(outtime, INTERVAL '2' HOUR)
    ) AS endtime
    , weight
    , weight_type
  from wt_stg3
)
-- if the intime for the patient is < the first charted daily weight
-- THEN we will have a "gap" at the start of their stay
-- to prevent this, we look for these gaps and backfill the first weight
-- this adds (153255-149657)=3598 rows, meaning this fix helps for up to 3598 stay_id
, wt_fix AS
(
  SELECT ie.stay_id
    -- we add a 2 hour "fuzziness" window
    , DATETIME_SUB(ie.intime, INTERVAL '2' HOUR) AS starttime
    , wt.starttime AS endtime
    , wt.weight
    , wt.weight_type
  from `physionet-data.mimic_icu.icustays` ie
  inner join
  -- the below subquery returns one row for each unique stay_id
  -- the row contains: the first starttime and the corresponding weight
  (
    SELECT wt1.stay_id, wt1.starttime, wt1.weight
    , weight_type
    , ROW_NUMBER() OVER (PARTITION BY wt1.stay_id ORDER BY wt1.starttime) AS rn
    FROM wt1
  ) wt
    ON  ie.stay_id = wt.stay_id
    AND wt.rn = 1
    and ie.intime < wt.starttime
)
-- add the backfill rows to the main weight table
SELECT
wt1.stay_id
, wt1.starttime
, wt1.endtime
, wt1.weight
, wt1.weight_type
FROM wt1
UNION ALL
SELECT
wt_fix.stay_id
, wt_fix.starttime
, wt_fix.endtime
, wt_fix.weight
, wt_fix.weight_type
FROM wt_fix;
"""

In [11]:
vital_sign_string = """
WITH vital_signs_cohort AS(
  SELECT vital.stay_id
    , AVG(vital.temperature) AS temperature_mean, AVG(vital.heart_rate) AS heart_rate_mean, AVG(vital.resp_rate) AS resp_rate_mean, AVG(vital.mbp) AS mbp_mean
  FROM `physionet-data.mimic_derived.vitalsign` vital
  LEFT JOIN `physionet-data.mimic_icu.icustays` ic USING (stay_id)
  WHERE vital.charttime BETWEEN DATETIME_TRUNC(ic.intime-interval'1'day, DAY) AND ic.outtime
  GROUP BY vital.stay_id
)
SELECT * FROM vital_signs_cohort
"""

In [12]:
lab_tests_string = """
WITH blood AS (
  SELECT ic.stay_id
    , AVG(hemoglobin) AS hemoglobin
    , AVG(platelet) AS platelet
    , AVG(wbc) AS wbc
  FROM `physionet-data.mimic_derived.complete_blood_count` cbc
  LEFT JOIN `physionet-data.mimic_icu.icustays` ic USING (hadm_id)
  WHERE cbc.charttime BETWEEN DATE_TRUNC(ic.intime-interval'1'day, DAY) AND ic.outtime
  GROUP BY ic.stay_id
)
, chem AS (
  SELECT ic.stay_id
    , AVG(sodium) AS sodium
    , AVG(potassium) AS potassium
    , AVG(bicarbonate) AS bicarbonate
    , AVG(chloride) AS chloride
    , AVG(bun) AS bun
    , AVG(creatinine) AS creatinine
  FROM `physionet-data.mimic_derived.chemistry` chem
  LEFT JOIN `physionet-data.mimic_icu.icustays` ic USING (hadm_id)
  WHERE chem.charttime BETWEEN DATE_TRUNC(ic.intime-interval'1'day, DAY) AND ic.outtime
  GROUP BY ic.stay_id
)
, bg AS (
  SELECT ic.stay_id
     , AVG(lactate) AS lactate
     , AVG(ph) AS ph
     , AVG(po2) AS po2
     , AVG(pco2) AS pco2
  FROM  `physionet-data.mimic_derived.bg` bg
  LEFT JOIN `physionet-data.mimic_icu.icustays` ic USING (hadm_id)
  WHERE bg.charttime BETWEEN DATE_TRUNC(ic.intime-interval'1'day, DAY) AND ic.outtime
  GROUP BY ic.stay_id
)
, logical_tested_index AS (
  SELECT ic.stay_id
    , AVG(CASE WHEN ntprobnp is not null THEN 1 ELSE 0 END) AS bnp
    , AVG(CASE WHEN troponin_t is not null THEN 1 ELSE 0 END)AS troponin
    , AVG(CASE WHEN ck_mb is not null THEN 1 ELSE 0 END) AS creatinine_kinase
  FROM `physionet-data.mimic_derived.cardiac_marker` cm
  LEFT JOIN `physionet-data.mimic_icu.icustays` ic USING (hadm_id)
  WHERE cm.charttime BETWEEN DATE_TRUNC(ic.intime-interval'1'day, DAY) AND ic.outtime
  GROUP BY ic.stay_id
)
, lab_tests AS (
  SELECT ic.stay_id
    , blood.hemoglobin, blood.platelet, blood.wbc
    , chem.sodium, chem.potassium, chem.bicarbonate, chem.chloride, chem.bun, chem.creatinine
    , bg.lactate, bg.ph, bg.po2, bg.pco2
    , lti.bnp, lti.troponin, lti.creatinine_kinase
  
  FROM `physionet-data.mimic_icu.icustays` ic
  LEFT JOIN blood USING (stay_id)
  LEFT JOIN chem USING (stay_id)
  LEFT JOIN bg USING (stay_id)
  LEFT JOIN logical_tested_index lti USING (stay_id)
)
SELECT * FROM lab_tests
"""

In [67]:
score_SAPS_string = """
-- ------------------------------------------------------------------
-- Title: Simplified Acute Physiology Score II (SAPS II)
-- This query extracts the simplified acute physiology score II.
-- This score is a measure of patient severity of illness.
-- The score is calculated on the first day of each ICU patients' stay.
-- ------------------------------------------------------------------


-- Variables used in SAPS II:
--  Age, GCS
--  VITALS: Heart rate, systolic blood pressure, temperature
--  FLAGS: ventilation/cpap
--  IO: urine output
--  LABS: PaO2/FiO2 ratio, blood urea nitrogen, WBC, potassium, sodium, HCO3
with co as
(
    select 
        subject_id
        , hadm_id
        , stay_id
        , intime AS starttime
        , DATETIME_ADD(intime, INTERVAL '24' HOUR) AS endtime
    from `physionet-data.mimic_icu.icustays` ie
)
, cpap as
(
  select 
    co.subject_id
    , co.stay_id
    , GREATEST(min(DATETIME_SUB(charttime, INTERVAL '1' HOUR)), co.starttime) as starttime
    , LEAST(max(DATETIME_ADD(charttime, INTERVAL '4' HOUR)), co.endtime) as endtime
    , max(case when REGEXP_CONTAINS(lower(ce.value), '(cpap mask|bipap)') then 1 else 0 end) as cpap
  from co
  inner join `physionet-data.mimic_icu.chartevents` ce
    on co.stay_id = ce.stay_id
    and ce.charttime > co.starttime
    and ce.charttime <= co.endtime
  where ce.itemid = 226732
  and REGEXP_CONTAINS(lower(ce.value), '(cpap mask|bipap)')
  group by co.subject_id, co.stay_id, co.starttime,co.endtime
)

-- extract a flag for surgical service
-- this combined with "elective" from admissions table defines elective/non-elective surgery
, surgflag as
(
  select adm.hadm_id
    , case when lower(curr_service) like '%surg%' then 1 else 0 end as surgical
    , ROW_NUMBER() over
    (
      PARTITION BY adm.HADM_ID
      ORDER BY TRANSFERTIME
    ) as serviceOrder
  from `physionet-data.mimic_core.admissions` adm
  left join `physionet-data.mimic_hosp.services` se
    on adm.hadm_id = se.hadm_id
)
-- icd-9 diagnostic codes are our best source for comorbidity information
-- unfortunately, they are technically a-causal
-- however, this shouldn't matter too much for the SAPS II comorbidities
, comorb as
(
select hadm_id
-- these are slightly different than elixhauser comorbidities, but based on them
-- they include some non-comorbid ICD-9 codes (e.g. 20302, relapse of multiple myeloma)
  , MAX(CASE
    WHEN icd_version = 9 AND SUBSTR(icd_code, 1, 3) BETWEEN '042' AND '044'
      THEN 1
    WHEN icd_version = 10 AND SUBSTR(icd_code, 1, 3) BETWEEN 'B20' AND 'B22' THEN 1
    WHEN icd_version = 10 AND SUBSTR(icd_code, 1, 3) = 'B24' THEN 1
  ELSE 0 END) AS aids  /* HIV and AIDS */
  , MAX(
    CASE WHEN icd_version = 9 THEN
      CASE
        WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20000' AND '20238' THEN 1 -- lymphoma
        WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20240' AND '20248' THEN 1 -- leukemia
        WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20250' AND '20302' THEN 1 -- lymphoma
        WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20310' AND '20312' THEN 1 -- leukemia
        WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20302' AND '20382' THEN 1 -- lymphoma
        WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20400' AND '20522' THEN 1 -- chronic leukemia
        WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20580' AND '20702' THEN 1 -- other myeloid leukemia
        WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20720' AND '20892' THEN 1 -- other myeloid leukemia
        WHEN SUBSTR(icd_code, 1, 4) IN ('2386', '2733') then 1 -- lymphoma
      ELSE 0 END
    WHEN icd_version = 10 AND SUBSTR(icd_code, 1, 3) BETWEEN 'C81' AND 'C96' THEN 1
  ELSE 0 END) as hem
  , MAX(CASE
    WHEN icd_version = 9 THEN
      CASE
      WHEN SUBSTR(icd_code, 1, 4) BETWEEN '1960' AND '1991' THEN 1
      WHEN SUBSTR(icd_code, 1, 5) BETWEEN '20970' AND '20975' THEN 1
      WHEN SUBSTR(icd_code, 1, 5) IN ('20979', '78951') THEN 1
      ELSE 0 END
    WHEN icd_version = 10 AND SUBSTR(icd_code, 1, 3) BETWEEN 'C77' AND 'C79' THEN 1
    WHEN icd_version = 10 AND SUBSTR(icd_code, 1, 4) = 'C800' THEN 1
    ELSE 0 END) as mets      /* Metastatic cancer */
    from `physionet-data.mimic_hosp.diagnoses_icd`
  group by hadm_id
)

, pafi1 as
(
  -- join blood gas to ventilation durations to determine if patient was vent
  -- also join to cpap table for the same purpose
  select 
    co.stay_id
  , bg.charttime
  , pao2fio2ratio AS PaO2FiO2
  , case when vd.stay_id is not null then 1 else 0 end as vent
  , case when cp.subject_id is not null then 1 else 0 end as cpap
  from co
  LEFT JOIN `physionet-data.mimic_derived.bg` bg
    ON co.subject_id = bg.subject_id
    AND bg.specimen = 'ART.'
    AND bg.charttime > co.starttime
    AND bg.charttime <= co.endtime
  left join `physionet-data.mimic_derived.ventilation` vd
    on co.stay_id = vd.stay_id
    and bg.charttime > vd.starttime
    and bg.charttime <= vd.endtime
    and vd.ventilation_status = 'InvasiveVent'
  left join cpap cp
    on bg.subject_id = cp.subject_id
    and bg.charttime > cp.starttime
    and bg.charttime <= cp.endtime
)
, pafi2 as
(
  -- get the minimum PaO2/FiO2 ratio *only for ventilated/cpap patients*
  select stay_id
  , min(PaO2FiO2) as PaO2FiO2_vent_min
  from pafi1
  where vent = 1 or cpap = 1
  group by stay_id
)

, gcs AS
(
    select co.stay_id
    , MIN(gcs.gcs) AS mingcs
    FROM co
    left join `physionet-data.mimic_derived.gcs` gcs
    ON co.stay_id = gcs.stay_id
    AND co.starttime < gcs.charttime
    AND gcs.charttime <= co.endtime
    GROUP BY co.stay_id
)

, vital AS 
(
    SELECT 
        co.stay_id
      , MIN(vital.heart_rate) AS heartrate_min
      , MAX(vital.heart_rate) AS heartrate_max
      , MIN(vital.sbp) AS sysbp_min
      , MAX(vital.sbp) AS sysbp_max
      , MIN(vital.temperature) AS tempc_min
      , MAX(vital.temperature) AS tempc_max
    FROM co
    left join `physionet-data.mimic_derived.vitalsign` vital
      on co.subject_id = vital.subject_id
      AND co.starttime < vital.charttime
      AND co.endtime >= vital.charttime
    GROUP BY co.stay_id
)
, uo AS
(
    SELECT 
        co.stay_id
      , SUM(uo.urineoutput) as urineoutput
    FROM co
    left join `physionet-data.mimic_derived.urine_output` uo
      on co.stay_id = uo.stay_id
      AND co.starttime < uo.charttime
      AND co.endtime >= uo.charttime
    GROUP BY co.stay_id
)
, labs AS
(
    SELECT 
        co.stay_id
      , MIN(labs.bun) AS bun_min
      , MAX(labs.bun) AS bun_max
      , MIN(labs.potassium) AS potassium_min
      , MAX(labs.potassium) AS potassium_max
      , MIN(labs.sodium) AS sodium_min
      , MAX(labs.sodium) AS sodium_max
      , MIN(labs.bicarbonate) AS bicarbonate_min
      , MAX(labs.bicarbonate) AS bicarbonate_max               
    FROM co
    left join `physionet-data.mimic_derived.chemistry` labs
      on co.subject_id = labs.subject_id
      AND co.starttime < labs.charttime
      AND co.endtime >= labs.charttime
    group by co.stay_id
)
, cbc AS
(
    SELECT 
        co.stay_id
      , MIN(cbc.wbc) AS wbc_min
      , MAX(cbc.wbc) AS wbc_max  
    FROM co
    LEFT JOIN `physionet-data.mimic_derived.complete_blood_count` cbc
      ON co.subject_id = cbc.subject_id
      AND co.starttime < cbc.charttime
      AND co.endtime >= cbc.charttime
    GROUP BY co.stay_id
)
, enz AS
(
    SELECT 
        co.stay_id
      , MIN(enz.bilirubin_total) AS bilirubin_min
      , MAX(enz.bilirubin_total) AS bilirubin_max  
    FROM co
    LEFT JOIN `physionet-data.mimic_derived.enzyme` enz
      ON co.subject_id = enz.subject_id
      AND co.starttime < enz.charttime
      AND co.endtime >= enz.charttime
    GROUP BY co.stay_id
)

, cohort as
(
select 
    ie.subject_id, ie.hadm_id, ie.stay_id
      , ie.intime
      , ie.outtime
      , va.age
      , co.starttime
      , co.endtime
    
      , vital.heartrate_max
      , vital.heartrate_min
      , vital.sysbp_max
      , vital.sysbp_min
      , vital.tempc_max
      , vital.tempc_min

      -- this value is non-null iff the patient is on vent/cpap
      , pf.PaO2FiO2_vent_min

      , uo.urineoutput

      , labs.bun_min
      , labs.bun_max
      , cbc.wbc_min
      , cbc.wbc_max
      , labs.potassium_min
      , labs.potassium_max
      , labs.sodium_min
      , labs.sodium_max
      , labs.bicarbonate_min
      , labs.bicarbonate_max
    
      , enz.bilirubin_min
      , enz.bilirubin_max

      , gcs.mingcs

      , comorb.AIDS
      , comorb.HEM
      , comorb.METS

      , case
          when adm.ADMISSION_TYPE = 'ELECTIVE' and sf.surgical = 1
            then 'ScheduledSurgical'
          when adm.ADMISSION_TYPE != 'ELECTIVE' and sf.surgical = 1
            then 'UnscheduledSurgical'
          else 'Medical'
        end as AdmissionType


from `physionet-data.mimic_icu.icustays` ie
inner join `physionet-data.mimic_core.admissions` adm
  on ie.hadm_id = adm.hadm_id
LEFT JOIN `physionet-data.mimic_derived.age` va
  on ie.hadm_id = va.hadm_id
inner join co
  on ie.stay_id = co.stay_id
    
-- join to above views
left join pafi2 pf
  on ie.stay_id = pf.stay_id
left join surgflag sf
  on adm.hadm_id = sf.hadm_id and sf.serviceOrder = 1
left join comorb
  on ie.hadm_id = comorb.hadm_id

-- join to custom tables to get more data....
left join gcs gcs
  on ie.stay_id = gcs.stay_id
left join vital
  on ie.stay_id = vital.stay_id
left join uo
  on ie.stay_id = uo.stay_id
left join labs
  on ie.stay_id = labs.stay_id
left join cbc
  on ie.stay_id = cbc.stay_id
left join enz
  on ie.stay_id = enz.stay_id
)
, scorecomp as
(
select
  cohort.*
  -- Below code calculates the component scores needed for SAPS
  , case
      when age is null then null
      when age <  40 then 0
      when age <  60 then 7
      when age <  70 then 12
      when age <  75 then 15
      when age <  80 then 16
      when age >= 80 then 18
    end as age_score

  , case
      when heartrate_max is null then null
      when heartrate_min <   40 then 11
      when heartrate_max >= 160 then 7
      when heartrate_max >= 120 then 4
      when heartrate_min  <  70 then 2
      when  heartrate_max >= 70 and heartrate_max < 120
        and heartrate_min >= 70 and heartrate_min < 120
      then 0
    end as hr_score

  , case
      when  sysbp_min is null then null
      when  sysbp_min <   70 then 13
      when  sysbp_min <  100 then 5
      when  sysbp_max >= 200 then 2
      when  sysbp_max >= 100 and sysbp_max < 200
        and sysbp_min >= 100 and sysbp_min < 200
        then 0
    end as sysbp_score

  , case
      when tempc_max is null then null
      when tempc_max >= 39.0 then 3
      when tempc_min <  39.0 then 0
    end as temp_score

  , case
      when PaO2FiO2_vent_min is null then null
      when PaO2FiO2_vent_min <  100 then 11
      when PaO2FiO2_vent_min <  200 then 9
      when PaO2FiO2_vent_min >= 200 then 6
    end as PaO2FiO2_score

  , case
      when UrineOutput is null then null
      when UrineOutput <   500.0 then 11
      when UrineOutput <  1000.0 then 4
      when UrineOutput >= 1000.0 then 0
    end as uo_score

  , case
      when bun_max is null then null
      when bun_max <  28.0 then 0
      when bun_max <  84.0 then 6
      when bun_max >= 84.0 then 10
    end as bun_score

  , case
      when wbc_max is null then null
      when wbc_min <   1.0 then 12
      when wbc_max >= 20.0 then 3
      when wbc_max >=  1.0 and wbc_max < 20.0
       and wbc_min >=  1.0 and wbc_min < 20.0
        then 0
    end as wbc_score

  , case
      when potassium_max is null then null
      when potassium_min <  3.0 then 3
      when potassium_max >= 5.0 then 3
      when potassium_max >= 3.0 and potassium_max < 5.0
       and potassium_min >= 3.0 and potassium_min < 5.0
        then 0
      end as potassium_score

  , case
      when sodium_max is null then null
      when sodium_min  < 125 then 5
      when sodium_max >= 145 then 1
      when sodium_max >= 125 and sodium_max < 145
       and sodium_min >= 125 and sodium_min < 145
        then 0
      end as sodium_score

  , case
      when bicarbonate_max is null then null
      when bicarbonate_min <  15.0 then 5
      when bicarbonate_min <  20.0 then 3
      when bicarbonate_max >= 20.0
       and bicarbonate_min >= 20.0
          then 0
      end as bicarbonate_score

  , case
      when bilirubin_max is null then null
      when bilirubin_max  < 4.0 then 0
      when bilirubin_max  < 6.0 then 4
      when bilirubin_max >= 6.0 then 9
      end as bilirubin_score

   , case
      when mingcs is null then null
        when mingcs <  3 then null -- erroneous value/on trach
        when mingcs <  6 then 26
        when mingcs <  9 then 13
        when mingcs < 11 then 7
        when mingcs < 14 then 5
        when mingcs >= 14
         and mingcs <= 15
          then 0
        end as gcs_score

    , case
        when AIDS = 1 then 17
        when HEM  = 1 then 10
        when METS = 1 then 9
        else 0
      end as comorbidity_score

    , case
        when AdmissionType = 'ScheduledSurgical' then 0
        when AdmissionType = 'Medical' then 6
        when AdmissionType = 'UnscheduledSurgical' then 8
        else null
      end as admissiontype_score

from cohort
)
-- Calculate SAPS II here so we can use it in the probability calculation below
, score as
(
  select s.*
  -- coalesce statements impute normal score of zero if data element is missing
  , coalesce(age_score,0)
  + coalesce(hr_score,0)
  + coalesce(sysbp_score,0)
  + coalesce(temp_score,0)
  + coalesce(PaO2FiO2_score,0)
  + coalesce(uo_score,0)
  + coalesce(bun_score,0)
  + coalesce(wbc_score,0)
  + coalesce(potassium_score,0)
  + coalesce(sodium_score,0)
  + coalesce(bicarbonate_score,0)
  + coalesce(bilirubin_score,0)
  + coalesce(gcs_score,0)
  + coalesce(comorbidity_score,0)
  + coalesce(admissiontype_score,0)
    as SAPSII
  from scorecomp s
)
select s.subject_id, s.hadm_id, s.stay_id
, s.starttime
, s.endtime
, sapsii
, 1 / (1 + exp(- (-7.7631 + 0.0737*(SAPSII) + 0.9971*(ln(SAPSII + 1))) )) as sapsii_prob
, age_score
, hr_score
, sysbp_score
, temp_score
, PaO2FiO2_score
, uo_score
, bun_score
, wbc_score
, potassium_score
, sodium_score
, bicarbonate_score
, bilirubin_score
, gcs_score
, comorbidity_score
, admissiontype_score
from score s
"""

In [68]:
score_SOFA_string = """
-- ------------------------------------------------------------------
-- Title: Sequential Organ Failure Assessment (SOFA)
-- This query extracts the sequential organ failure assessment (formally: sepsis-related organ failure assessment).
-- This score is a measure of organ failure for patients in the ICU.
-- The score is calculated for **every hour** of the patient's ICU stay.
-- However, as the calculation window is 24 hours, care should be taken when
-- using the score before the end of the first day, as the data window is limited.
-- ------------------------------------------------------------------

-- Variables used in SOFA:
--  GCS, MAP, FiO2, Ventilation status (sourced FROM `physionet-data.mimic_icu.chartevents`)
--  Creatinine, Bilirubin, FiO2, PaO2, Platelets (sourced FROM `physionet-data.mimic_icu.labevents`)
--  Dopamine, Dobutamine, Epinephrine, Norepinephrine (sourced FROM `physionet-data.mimic_icu.inputevents_mv` and INPUTEVENTS_CV)
--  Urine output (sourced from OUTPUTEVENTS)

-- generate a row for every hour the patient was in the ICU
-- here, we generate a starttime/endtime for every hour of the patient's ICU stay
-- all of our joins to data will use these times to extract data pertinent to only that hour
WITH co AS
(
  select ih.stay_id, ie.hadm_id
  , hr
  -- start/endtime can be used to filter to values within this hour
  , DATETIME_SUB(ih.endtime, INTERVAL '1' HOUR) AS starttime
  , ih.endtime
  from `physionet-data.mimic_derived.icustay_hourly` ih
  INNER JOIN `physionet-data.mimic_icu.icustays` ie
    ON ih.stay_id = ie.stay_id
)
, pafi as
(
  -- join blood gas to ventilation durations to determine if patient was vent
  select ie.stay_id
  , bg.charttime
  -- because pafi has an interaction between vent/PaO2:FiO2, we need two columns for the score
  -- it can happen that the lowest unventilated PaO2/FiO2 is 68, but the lowest ventilated PaO2/FiO2 is 120
  -- in this case, the SOFA score is 3, *not* 4.
  , case when vd.stay_id is null then pao2fio2ratio else null end pao2fio2ratio_novent
  , case when vd.stay_id is not null then pao2fio2ratio else null end pao2fio2ratio_vent
  FROM `physionet-data.mimic_icu.icustays` ie
  inner join `physionet-data.mimic_derived.bg` bg
    on ie.subject_id = bg.subject_id
  left join `physionet-data.mimic_derived.ventilation` vd
    on ie.stay_id = vd.stay_id
    and bg.charttime >= vd.starttime
    and bg.charttime <= vd.endtime
    and vd.ventilation_status = 'InvasiveVent'
  WHERE specimen = 'ART.'
)
, vs AS
(
    
  select co.stay_id, co.hr
  -- vitals
  , min(vs.mbp) as meanbp_min
  from co
  left join `physionet-data.mimic_derived.vitalsign` vs
    on co.stay_id = vs.stay_id
    and co.starttime < vs.charttime
    and co.endtime >= vs.charttime
  group by co.stay_id, co.hr
)
, gcs AS
(
  select co.stay_id, co.hr
  -- gcs
  , min(gcs.gcs) as gcs_min
  from co
  left join `physionet-data.mimic_derived.gcs` gcs
    on co.stay_id = gcs.stay_id
    and co.starttime < gcs.charttime
    and co.endtime >= gcs.charttime
  group by co.stay_id, co.hr
)
, bili AS
(
  select co.stay_id, co.hr
  , max(enz.bilirubin_total) as bilirubin_max
  from co
  left join `physionet-data.mimic_derived.enzyme` enz
    on co.hadm_id = enz.hadm_id
    and co.starttime < enz.charttime
    and co.endtime >= enz.charttime
  group by co.stay_id, co.hr
)
, cr AS
(
  select co.stay_id, co.hr
  , max(chem.creatinine) as creatinine_max
  from co
  left join `physionet-data.mimic_derived.chemistry` chem
    on co.hadm_id = chem.hadm_id
    and co.starttime < chem.charttime
    and co.endtime >= chem.charttime
  group by co.stay_id, co.hr
)
, plt AS
(
  select co.stay_id, co.hr
  , min(cbc.platelet) as platelet_min
  from co
  left join `physionet-data.mimic_derived.complete_blood_count` cbc
    on co.hadm_id = cbc.hadm_id
    and co.starttime < cbc.charttime
    and co.endtime >= cbc.charttime
  group by co.stay_id, co.hr
)
, pf AS
(
  select co.stay_id, co.hr
  , min(pafi.pao2fio2ratio_novent) AS pao2fio2ratio_novent
  , min(pafi.pao2fio2ratio_vent) AS pao2fio2ratio_vent
  from co
  -- bring in blood gases that occurred during this hour
  left join pafi
    on co.stay_id = pafi.stay_id
    and co.starttime < pafi.charttime
    and co.endtime  >= pafi.charttime
  group by co.stay_id, co.hr
)
-- sum uo separately to prevent duplicating values
, uo as
(
  select co.stay_id, co.hr
  -- uo
  , MAX(
      CASE WHEN uo.uo_tm_24hr >= 22 AND uo.uo_tm_24hr <= 30
          THEN uo.urineoutput_24hr / uo.uo_tm_24hr * 24
  END) as uo_24hr
  from co
  left join `physionet-data.mimic_derived.urine_output_rate` uo
    on co.stay_id = uo.stay_id
    and co.starttime < uo.charttime
    and co.endtime >= uo.charttime
  group by co.stay_id, co.hr
)
-- collapse vasopressors into 1 row per hour
-- also ensures only 1 row per chart time
, vaso AS
(
    SELECT 
        co.stay_id
        , co.hr
        , MAX(epi.vaso_rate) as rate_epinephrine
        , MAX(nor.vaso_rate) as rate_norepinephrine
        , MAX(dop.vaso_rate) as rate_dopamine
        , MAX(dob.vaso_rate) as rate_dobutamine
    FROM co
    LEFT JOIN `physionet-data.mimic_derived.epinephrine` epi
        on co.stay_id = epi.stay_id
        and co.endtime > epi.starttime
        and co.endtime <= epi.endtime
    LEFT JOIN `physionet-data.mimic_derived.norepinephrine` nor
        on co.stay_id = nor.stay_id
        and co.endtime > nor.starttime
        and co.endtime <= nor.endtime
    LEFT JOIN `physionet-data.mimic_derived.dopamine` dop
        on co.stay_id = dop.stay_id
        and co.endtime > dop.starttime
        and co.endtime <= dop.endtime
    LEFT JOIN `physionet-data.mimic_derived.dobutamine` dob
        on co.stay_id = dob.stay_id
        and co.endtime > dob.starttime
        and co.endtime <= dob.endtime
    WHERE epi.stay_id IS NOT NULL
    OR nor.stay_id IS NOT NULL
    OR dop.stay_id IS NOT NULL
    OR dob.stay_id IS NOT NULL
    GROUP BY co.stay_id, co.hr
)
, scorecomp as
(
  select
      co.stay_id
    , co.hr
    , co.starttime, co.endtime
    , pf.pao2fio2ratio_novent
    , pf.pao2fio2ratio_vent
    , vaso.rate_epinephrine
    , vaso.rate_norepinephrine
    , vaso.rate_dopamine
    , vaso.rate_dobutamine
    , vs.meanbp_min
    , gcs.gcs_min
    -- uo
    , uo.uo_24hr
    -- labs
    , bili.bilirubin_max
    , cr.creatinine_max
    , plt.platelet_min
  from co
  left join vs
    on co.stay_id = vs.stay_id
    and co.hr = vs.hr
  left join gcs
    on co.stay_id = gcs.stay_id
    and co.hr = gcs.hr
  left join bili
    on co.stay_id = bili.stay_id
    and co.hr = bili.hr
  left join cr
    on co.stay_id = cr.stay_id
    and co.hr = cr.hr
  left join plt
    on co.stay_id = plt.stay_id
    and co.hr = plt.hr
  left join pf
    on co.stay_id = pf.stay_id
    and co.hr = pf.hr
  left join uo
    on co.stay_id = uo.stay_id
    and co.hr = uo.hr
  left join vaso
    on co.stay_id = vaso.stay_id
    and co.hr = vaso.hr
)
, scorecalc as
(
  -- Calculate the final score
  -- note that if the underlying data is missing, the component is null
  -- eventually these are treated as 0 (normal), but knowing when data is missing is useful for debugging
  select scorecomp.*
  -- Respiration
  , case
      when pao2fio2ratio_vent   < 100 then 4
      when pao2fio2ratio_vent   < 200 then 3
      when pao2fio2ratio_novent < 300 then 2
      when pao2fio2ratio_vent   < 300 then 2
      when pao2fio2ratio_novent < 400 then 1
      when pao2fio2ratio_vent   < 400 then 1
      when coalesce(pao2fio2ratio_vent, pao2fio2ratio_novent) is null then null
      else 0
    end as respiration

  -- Coagulation
  , case
      when platelet_min < 20  then 4
      when platelet_min < 50  then 3
      when platelet_min < 100 then 2
      when platelet_min < 150 then 1
      when platelet_min is null then null
      else 0
    end as coagulation

  -- Liver
  , case
      -- Bilirubin checks in mg/dL
        when bilirubin_max >= 12.0 then 4
        when bilirubin_max >= 6.0  then 3
        when bilirubin_max >= 2.0  then 2
        when bilirubin_max >= 1.2  then 1
        when bilirubin_max is null then null
        else 0
      end as liver

  -- Cardiovascular
  , case
      when rate_dopamine > 15 or rate_epinephrine >  0.1 or rate_norepinephrine >  0.1 then 4
      when rate_dopamine >  5 or rate_epinephrine <= 0.1 or rate_norepinephrine <= 0.1 then 3
      when rate_dopamine >  0 or rate_dobutamine > 0 then 2
      when meanbp_min < 70 then 1
      when coalesce(meanbp_min, rate_dopamine, rate_dobutamine, rate_epinephrine, rate_norepinephrine) is null then null
      else 0
    end as cardiovascular

  -- Neurological failure (GCS)
  , case
      when (gcs_min >= 13 and gcs_min <= 14) then 1
      when (gcs_min >= 10 and gcs_min <= 12) then 2
      when (gcs_min >=  6 and gcs_min <=  9) then 3
      when  gcs_min <   6 then 4
      when  gcs_min is null then null
      else 0
    end as cns

  -- Renal failure - high creatinine or low urine output
  , case
    when (creatinine_max >= 5.0) then 4
    when uo_24hr < 200 then 4
    when (creatinine_max >= 3.5 and creatinine_max < 5.0) then 3
    when uo_24hr < 500 then 3
    when (creatinine_max >= 2.0 and creatinine_max < 3.5) then 2
    when (creatinine_max >= 1.2 and creatinine_max < 2.0) then 1
    when coalesce (uo_24hr, creatinine_max) is null then null
    else 0 
  end as renal
  from scorecomp
)
, score_final as
(
  select s.*
    -- Combine all the scores to get SOFA
    -- Impute 0 if the score is missing
   -- the window function takes the max over the last 24 hours
    , coalesce(
        MAX(respiration) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as respiration_24hours
     , coalesce(
         MAX(coagulation) OVER (PARTITION BY stay_id ORDER BY HR
         ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
        ,0) as coagulation_24hours
    , coalesce(
        MAX(liver) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as liver_24hours
    , coalesce(
        MAX(cardiovascular) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as cardiovascular_24hours
    , coalesce(
        MAX(cns) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as cns_24hours
    , coalesce(
        MAX(renal) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as renal_24hours

    -- sum together data for final SOFA
    , coalesce(
        MAX(respiration) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
         MAX(coagulation) OVER (PARTITION BY stay_id ORDER BY HR
         ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
        MAX(liver) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
        MAX(cardiovascular) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
        MAX(cns) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
        MAX(renal) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
    as sofa_24hours
  from scorecalc s
  WINDOW W as
  (
    PARTITION BY stay_id
    ORDER BY hr
    ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING
  )
)
select * from score_final
where hr >= 0
"""

In [82]:
# Quering from saved table (change FROM `tablename` to your set name here)
score_SOFA = """
SELECT stay_id,sofa_24hours FROM `mit-midodrine.physionet.sofa`
"""

In [13]:
population_data = (
    bqclient.query(population)
    .result()
    .to_dataframe()
)

## Building the cohort table (consisting of only ICU patients)
1. Include adults (>= 18 years' old)
2. Include the first ICU admission of each patient
3. Include patients whose ICU care is 'MICU' or 'SICU'
4. Exclude those who accepted midodrine more than one day before icu care or after icu care (have been done before) 

Note: midodrine_inlcude is just opposite of midodrine_exclude so can use either of the 2 for this

In [14]:
cohort_data = (
    bqclient.query(population[:-25]+cohort_string)
    .result()
    .to_dataframe()
)

In [15]:
cohort_data

,stay_id,first_careunit,intime,outtime,subject_id,hadm_id,gender,dod,admittime,dischtime,...,hospital_expire_flag,icu_intime,icu_outtime,los_icu,icu_order,drug_starttime,midodrine_exclude,midodrine_include,avg_doses_per_24_hrs,drug
0,30219644,Trauma SICU (TSICU),2134-03-20 07:33:58,2134-03-21 18:03:57,12514289,20173644,M,None,2134-03-19 19:21:00,2134-03-28 14:25:00,...,0,2134-03-20 07:33:58,2134-03-21 18:03:57,1.46,1,NaT,0,1,1.837838,1
1,37081269,Medical Intensive Care Unit (MICU),2169-05-29 23:11:55,2169-05-30 09:51:26,12704088,28485732,F,None,2169-05-29 21:01:00,2169-06-02 16:30:00,...,0,2169-05-29 23:11:55,2169-05-30 09:51:26,0.42,1,NaT,0,1,1.375000,1
2,37218323,Medical/Surgical Intensive Care Unit (MICU/SICU),2138-12-31 15:41:00,2139-01-02 14:30:51,13243686,23194106,F,None,2138-12-31 13:25:00,2139-01-03 15:16:00,...,0,2138-12-31 15:41:00,2139-01-02 14:30:51,1.96,1,NaT,0,1,1.600000,1
3,35208530,Medical Intensive Care Unit (MICU),2152-08-14 14:28:00,2152-08-15 01:01:19,13382937,25110144,F,2152-08-14,2152-08-14 12:59:00,2152-08-14 01:00:00,...,1,2152-08-14 14:28:00,2152-08-15 01:01:19,0.46,1,NaT,0,1,1.384615,1
4,36716855,Medical/Surgical Intensive Care Unit (MICU/SICU),2142-05-02 18:24:23,2142-05-04 18:27:51,14450311,28257006,F,None,2142-04-30 02:04:00,2142-05-12 18:45:00,...,0,2142-05-02 18:24:23,2142-05-04 18:27:51,2.00,1,NaT,0,1,1.269231,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35295,32060452,Surgical Intensive Care Unit (SICU),2186-12-28 23:55:05,2187-01-03 18:39:27,15675265,28135573,M,2187-01-13,2186-12-20 16:19:00,2187-01-11 14:15:00,...,0,2186-12-28 23:55:05,2187-01-03 18:39:27,5.79,1,NaT,0,1,1.604651,1
35296,32916697,Surgical Intensive Care Unit (SICU),2162-02-12 11:35:00,2162-02-13 21:03:52,16223774,22570779,F,2163-02-27,2162-02-12 10:49:00,2162-02-17 11:53:00,...,0,2162-02-12 11:35:00,2162-02-13 21:03:52,1.42,1,NaT,0,1,1.326531,1
35297,37643192,Medical/Surgical Intensive Care Unit (MICU/SICU),2196-03-08 18:12:02,2196-03-10 13:09:16,19365784,25478150,M,None,2196-03-08 16:20:00,2196-03-16 12:50:00,...,0,2196-03-08 18:12:02,2196-03-10 13:09:16,1.79,1,NaT,0,1,1.444444,1
35298,31045933,Trauma SICU (TSICU),2125-09-16 23:52:00,2125-09-19 09:12:48,13791052,26747635,F,None,2125-09-16 22:16:00,2125-09-21 14:30:00,...,0,2125-09-16 23:52:00,2125-09-19 09:12:48,2.42,1,NaT,0,1,2.068966,1


In [56]:
# Admission day
cohort_data['admission_day'] = pd.to_datetime(cohort_data['intime']).dt.day_name()
cohort_data

,stay_id,first_careunit,intime,outtime,subject_id,hadm_id,gender,dod,admittime,dischtime,...,icu_intime,icu_outtime,los_icu,icu_order,drug_starttime,midodrine_exclude,midodrine_include,avg_doses_per_24_hrs,drug,admission_day
0,30219644,Trauma SICU (TSICU),2134-03-20 07:33:58,2134-03-21 18:03:57,12514289,20173644,M,None,2134-03-19 19:21:00,2134-03-28 14:25:00,...,2134-03-20 07:33:58,2134-03-21 18:03:57,1.46,1,NaT,0,1,1.837838,1,Saturday
1,37081269,Medical Intensive Care Unit (MICU),2169-05-29 23:11:55,2169-05-30 09:51:26,12704088,28485732,F,None,2169-05-29 21:01:00,2169-06-02 16:30:00,...,2169-05-29 23:11:55,2169-05-30 09:51:26,0.42,1,NaT,0,1,1.375000,1,Monday
2,37218323,Medical/Surgical Intensive Care Unit (MICU/SICU),2138-12-31 15:41:00,2139-01-02 14:30:51,13243686,23194106,F,None,2138-12-31 13:25:00,2139-01-03 15:16:00,...,2138-12-31 15:41:00,2139-01-02 14:30:51,1.96,1,NaT,0,1,1.600000,1,Wednesday
3,35208530,Medical Intensive Care Unit (MICU),2152-08-14 14:28:00,2152-08-15 01:01:19,13382937,25110144,F,2152-08-14,2152-08-14 12:59:00,2152-08-14 01:00:00,...,2152-08-14 14:28:00,2152-08-15 01:01:19,0.46,1,NaT,0,1,1.384615,1,Monday
4,36716855,Medical/Surgical Intensive Care Unit (MICU/SICU),2142-05-02 18:24:23,2142-05-04 18:27:51,14450311,28257006,F,None,2142-04-30 02:04:00,2142-05-12 18:45:00,...,2142-05-02 18:24:23,2142-05-04 18:27:51,2.00,1,NaT,0,1,1.269231,1,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35295,32060452,Surgical Intensive Care Unit (SICU),2186-12-28 23:55:05,2187-01-03 18:39:27,15675265,28135573,M,2187-01-13,2186-12-20 16:19:00,2187-01-11 14:15:00,...,2186-12-28 23:55:05,2187-01-03 18:39:27,5.79,1,NaT,0,1,1.604651,1,Thursday
35296,32916697,Surgical Intensive Care Unit (SICU),2162-02-12 11:35:00,2162-02-13 21:03:52,16223774,22570779,F,2163-02-27,2162-02-12 10:49:00,2162-02-17 11:53:00,...,2162-02-12 11:35:00,2162-02-13 21:03:52,1.42,1,NaT,0,1,1.326531,1,Friday
35297,37643192,Medical/Surgical Intensive Care Unit (MICU/SICU),2196-03-08 18:12:02,2196-03-10 13:09:16,19365784,25478150,M,None,2196-03-08 16:20:00,2196-03-16 12:50:00,...,2196-03-08 18:12:02,2196-03-10 13:09:16,1.79,1,NaT,0,1,1.444444,1,Tuesday
35298,31045933,Trauma SICU (TSICU),2125-09-16 23:52:00,2125-09-19 09:12:48,13791052,26747635,F,None,2125-09-16 22:16:00,2125-09-21 14:30:00,...,2125-09-16 23:52:00,2125-09-19 09:12:48,2.42,1,NaT,0,1,2.068966,1,Sunday


In [65]:
cohort_data.isnull().sum()

stay_id                     0
first_careunit              0
intime                      0
outtime                     0
subject_id                  0
hadm_id                     0
gender                      0
dod                     29209
admittime                   0
dischtime                   0
los_hospital                0
age                         0
ethnicity                   0
hospital_expire_flag        0
icu_intime                  0
icu_outtime                 0
los_icu                     0
icu_order                   0
drug_starttime          34490
midodrine_exclude           0
midodrine_include           0
avg_doses_per_24_hrs      116
drug                        0
admission_day               0
dtype: int64

## Extract the weight

In [16]:
weight_data = (
    bqclient.query(weight_string)
    .result()
    .to_dataframe()
)

In [17]:
weight_data

,stay_id,starttime,endtime,weight,weight_type
0,30708785,2189-12-10 18:13:00,2189-12-11 00:00:00,100.5,daily
1,30753165,2171-03-04 09:50:42,2171-03-05 13:00:00,75.0,daily
2,30969285,2154-03-03 22:08:54,2154-03-04 07:00:00,76.0,daily
3,31008267,2167-06-09 15:14:25,2167-06-10 00:00:00,51.8,daily
4,31201587,2129-03-06 21:48:43,2129-03-07 06:00:00,106.0,daily
...,...,...,...,...,...
287150,38889046,2170-07-09 17:37:48,2170-07-10 01:00:00,75.5,daily
287151,39408011,2120-10-08 02:45:52,2120-10-09 03:00:00,97.8,daily
287152,39832500,2206-04-10 07:44:08,2206-04-11 06:00:00,75.6,daily
287153,39952460,2121-09-28 20:04:14,2121-09-30 00:00:00,89.2,daily


## Extract the Comorbidities

In [18]:
comorbidities_data = (
    bqclient.query(comorbidities_string)
    .result()
    .to_dataframe()
)

In [19]:
comorbidities_data

,stay_id,CPD_or_COPD_rate,Maligancy_rate,mild_liver_rate,severe_liver_rate,Renal_rate,CAD_rate,AFIB_rate
0,34547665,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39289362,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32563675,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,34947848,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,37445058,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...
76535,39980385,1.0,0.0,0.0,0.0,1.0,1.0,1.0
76536,39985296,0.0,1.0,0.0,0.0,0.0,0.0,0.0
76537,39987031,1.0,0.0,0.0,0.0,1.0,0.0,1.0
76538,39989040,1.0,1.0,0.0,0.0,1.0,0.0,0.0


## Extract Vital Signs

In [20]:
vital_sign_data = (
    bqclient.query(vital_sign_string)
    .result()
    .to_dataframe()
)

In [21]:
vital_sign_data

,stay_id,temperature_mean,heart_rate_mean,resp_rate_mean,mbp_mean
0,38150079,36.652020,60.860377,19.847786,65.923208
1,36561068,37.480000,114.439024,31.317073,95.241379
2,37147644,38.168125,87.863158,24.336842,94.031579
3,34515335,36.739412,88.823529,23.157143,74.684932
4,38750362,37.081176,90.231707,25.573171,70.737805
...,...,...,...,...,...
76523,37830507,NaN,80.000000,25.000000,80.000000
76524,36143527,NaN,94.666667,26.333333,57.500000
76525,39275363,NaN,103.000000,28.000000,73.000000
76526,34387079,NaN,71.000000,31.000000,NaN


## Extract Lab Tests Results

In [22]:
lab_tests_data = (
    bqclient.query(lab_tests_string)
    .result()
    .to_dataframe()
)

In [23]:
lab_tests_data

,stay_id,hemoglobin,platelet,wbc,sodium,potassium,bicarbonate,chloride,bun,creatinine,lactate,ph,po2,pco2,bnp,troponin,creatinine_kinase
0,31751864,8.775000,272.750000,9.32500,140.000000,4.300000,23.500000,103.750000,42.500000,1.675000,NaN,NaN,NaN,NaN,0.000000,0.833333,0.666667
1,38690211,12.066667,221.166667,12.40000,138.600000,3.660000,22.400000,102.600000,21.833333,1.216667,NaN,NaN,NaN,NaN,0.000000,0.833333,1.000000
2,30571173,9.850000,163.250000,5.07500,137.000000,4.625000,32.250000,95.250000,30.750000,1.275000,NaN,NaN,NaN,NaN,0.200000,0.600000,0.600000
3,32423515,13.460000,167.600000,7.32000,142.375000,4.050000,23.875000,105.000000,11.125000,0.712500,NaN,NaN,NaN,NaN,0.000000,0.600000,1.000000
4,30399955,15.040000,163.200000,11.38000,136.500000,4.033333,23.333333,100.666667,13.500000,1.216667,NaN,NaN,NaN,NaN,0.000000,0.333333,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76535,33548341,13.000000,178.666667,11.00000,142.666667,3.566667,25.666667,108.000000,8.666667,0.700000,1.500000,7.490000,154.000000,36.000000,0.000000,0.666667,1.000000
76536,37904621,9.300000,463.500000,12.65000,139.833333,3.814286,26.333333,101.666667,18.500000,1.066667,NaN,7.428333,130.666667,40.666667,0.333333,0.666667,0.666667
76537,38109349,7.712500,262.125000,9.31875,136.666667,4.047619,28.190476,98.666667,18.571429,1.085714,2.070000,7.364706,50.352941,52.411765,0.666667,0.666667,0.666667
76538,39446578,10.370000,270.900000,31.28000,143.000000,4.000000,22.181818,108.818182,23.818182,1.490909,4.550000,7.300000,122.000000,49.500000,0.333333,0.666667,0.333333


## Extract SAPS Score

In [69]:
score_SAPS_data = (
    bqclient.query(score_SAPS_string)
    .result()
    .to_dataframe()
)

In [70]:
score_SAPS_data

,subject_id,hadm_id,stay_id,starttime,endtime,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,...,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,13307171,21128752,30576177,2145-11-22 00:54:00,2145-11-23 00:54:00,12,0.013110,0,2.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
1,15642792,27536715,37186192,2173-01-16 18:40:00,2173-01-17 18:40:00,61,0.700098,18,11.0,0.0,...,11.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,10,6
2,13735793,21147672,35366563,2168-05-20 09:52:00,2168-05-21 09:52:00,6,0.004584,0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
3,11084025,20933925,34413397,2130-07-30 11:03:00,2130-07-31 11:03:00,6,0.004584,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
4,18204932,29920194,39324365,2150-08-13 20:06:13,2150-08-14 20:06:13,11,0.011265,0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76535,14128850,29453195,36431046,2181-10-21 04:12:09,2181-10-22 04:12:09,60,0.680887,18,4.0,13.0,...,4.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0,6
76536,10066209,27826282,36514617,2121-07-04 01:00:33,2121-07-05 01:00:33,60,0.680887,18,2.0,5.0,...,4.0,6.0,0.0,0.0,0.0,3.0,0.0,7.0,0,6
76537,17577670,26822848,37055947,2153-10-02 13:21:00,2153-10-03 13:21:00,60,0.680887,18,2.0,5.0,...,0.0,6.0,12.0,0.0,1.0,0.0,NaN,0.0,10,6
76538,19001252,24531176,37707387,2140-10-25 01:54:00,2140-10-26 01:54:00,60,0.680887,16,0.0,5.0,...,0.0,0.0,0.0,3.0,1.0,3.0,0.0,26.0,0,6


## Extract SOFA Score

In [83]:
SOFA_data = (
    bqclient.query(score_SOFA)
    .result()
    .to_dataframe()
)

In [89]:
SOFA_data = SOFA_data.drop_duplicates(subset=['stay_id'])

,stay_id,sofa_24hours
0,30006983,15
15,30058917,15
23,30106638,16
29,30146534,16
39,30225882,16
...,...,...
6248974,35206016,11
6250457,35540367,11
6255557,35561369,11
6305159,38507218,12


In [90]:
SOFA_data

,stay_id,sofa_24hours
0,30006983,15
1,30006983,15
2,30006983,15
3,30006983,15
4,30006983,15
...,...,...
6358762,39918058,14
6358763,39918058,14
6358764,39918058,14
6358765,39918058,14


In [71]:
score_SOFA_data = (
    bqclient.query(score_SOFA_string)
    .result()
    .to_dataframe()
)

In [72]:
score_SOFA_data

,stay_id,hr,starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,30001148,0,2156-08-30 14:00:00,2156-08-30 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,4.0,NaN,0,0,0,0,4,0,4
1,30001148,1,2156-08-30 15:00:00,2156-08-30 16:00:00,300.0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,1,0,0,0,4,0,5
2,30001148,2,2156-08-30 16:00:00,2156-08-30 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,1,0,0,0,4,0,5
3,30001148,4,2156-08-30 18:00:00,2156-08-30 19:00:00,350.0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,1,0,0,0,4,0,5
4,30001148,5,2156-08-30 19:00:00,2156-08-30 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,NaN,1,0,0,1,4,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6358762,39999810,99,2115-12-05 04:00:00,2115-12-05 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,0,0,0,1,0,1
6358763,39999810,101,2115-12-05 06:00:00,2115-12-05 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,0,0,0,1,0,1
6358764,39999810,104,2115-12-05 09:00:00,2115-12-05 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,0,0,0,1,0,1
6358765,39999810,106,2115-12-05 11:00:00,2115-12-05 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,0,0,0,0,1,0,1


## Divide the Midodrine Group

In [59]:
cohort_mido = cohort_data[pd.notnull(cohort_data['drug_starttime'])]
cohort_mido.describe()

,stay_id,subject_id,hadm_id,los_hospital,age,hospital_expire_flag,los_icu,icu_order,midodrine_exclude,midodrine_include,avg_doses_per_24_hrs,drug
count,8.100000e+02,8.100000e+02,8.100000e+02,810.000000,810.000000,810.000000,810.000000,810.0,810.0,810.0,810.000000,810.0
mean,3.492571e+07,1.506504e+07,2.513436e+07,21.497531,62.777778,0.322222,10.107864,1.0,0.0,1.0,1.627904,0.0
std,2.896935e+06,2.836112e+06,2.970905e+06,18.881076,15.082796,0.467616,11.504315,0.0,0.0,0.0,0.291719,0.0
min,3.003771e+07,1.000892e+07,2.000173e+07,0.000000,18.000000,0.000000,0.130000,1.0,0.0,1.0,0.985075,0.0
25%,3.236288e+07,1.266337e+07,2.243435e+07,8.000000,53.000000,0.000000,2.830000,1.0,0.0,1.0,1.446547,0.0
50%,3.490289e+07,1.496978e+07,2.541225e+07,16.000000,63.000000,0.000000,5.830000,1.0,0.0,1.0,1.578947,0.0
75%,3.736149e+07,1.746826e+07,2.782586e+07,29.000000,73.000000,1.000000,13.410000,1.0,0.0,1.0,1.758242,0.0
max,3.999923e+07,1.999930e+07,2.999661e+07,193.000000,98.000000,1.000000,99.630000,1.0,0.0,1.0,4.653061,0.0


In [25]:
mido_weights = weight_data[(weight_data['stay_id'].isin(cohort_mido['stay_id'].values)) & (weight_data['weight_type']=='admit')]
mido_weights.describe()

,stay_id,weight
count,7.810000e+02,781.000000
mean,3.494998e+07,84.899872
std,2.910323e+06,26.257020
min,3.003771e+07,32.900000
25%,3.235555e+07,67.700000
50%,3.492028e+07,80.600000
75%,3.743279e+07,98.400000
max,3.999923e+07,230.000000


In [26]:
mido_vital_signs = vital_sign_data[vital_sign_data['stay_id'].isin(cohort_mido['stay_id'].values)]
mido_vital_signs.describe()

,stay_id,temperature_mean,heart_rate_mean,resp_rate_mean,mbp_mean
count,8.100000e+02,808.000000,810.000000,810.000000,810.000000
mean,3.492571e+07,36.790786,84.924612,19.286474,72.347926
std,2.896935e+06,0.384468,13.817980,3.628422,9.226079
min,3.003771e+07,35.220000,47.952830,10.192308,47.479167
25%,3.236288e+07,36.590974,75.063061,16.770041,66.657449
50%,3.490289e+07,36.777413,85.395760,19.100934,71.635066
75%,3.736149e+07,36.983953,94.941119,21.469442,77.035680
max,3.999923e+07,38.130000,124.914634,48.000000,124.362637


In [27]:
mido_labs = lab_tests_data[lab_tests_data['stay_id'].isin(cohort_mido['stay_id'].values)]
mido_labs.describe()

,stay_id,hemoglobin,platelet,wbc,sodium,potassium,bicarbonate,chloride,bun,creatinine,lactate,ph,po2,pco2,bnp,troponin,creatinine_kinase
count,8.100000e+02,805.000000,806.000000,806.000000,807.000000,807.000000,807.000000,807.000000,807.000000,807.000000,648.000000,684.000000,684.000000,684.000000,385.000000,385.000000,385.000000
mean,3.492571e+07,9.205536,161.222140,12.477541,137.729635,4.141906,22.478938,102.084150,39.560938,2.281318,2.415392,7.371469,97.063678,40.366431,0.118722,0.645947,0.713748
std,2.896935e+06,1.459199,108.758313,6.455160,5.080456,0.490859,4.525356,6.289177,27.033225,1.848305,1.666055,0.066215,46.085147,8.619284,0.290957,0.442502,0.399717
min,3.003771e+07,6.550000,13.285714,0.733333,115.071429,3.000000,10.277778,73.500000,3.800000,0.106667,0.400000,7.122857,19.000000,20.250000,0.000000,0.000000,0.000000
25%,3.236288e+07,8.108333,75.872024,7.927841,135.000000,3.806232,19.430769,98.178125,18.226959,0.869035,1.495161,7.328807,64.107143,34.816176,0.000000,0.000000,0.500000
50%,3.490289e+07,8.900000,135.444444,11.280000,138.000000,4.100000,22.435484,102.250000,33.714286,1.777778,2.000000,7.375329,92.679438,39.038462,0.000000,1.000000,1.000000
75%,3.736149e+07,9.976923,218.960648,15.612987,141.000000,4.396667,25.111280,105.736111,53.507143,3.009809,2.725893,7.415604,116.561441,44.458333,0.000000,1.000000,1.000000
max,3.999923e+07,15.291667,796.000000,67.900000,154.454545,6.360000,41.684211,121.857143,177.500000,12.300000,16.456250,7.570000,389.000000,81.254902,1.000000,1.000000,1.000000


In [28]:
mido_comorbidities = comorbidities_data[comorbidities_data['stay_id'].isin(cohort_mido['stay_id'].values)]
mido_comorbidities.describe()

,stay_id,CPD_or_COPD_rate,Maligancy_rate,mild_liver_rate,severe_liver_rate,Renal_rate,CAD_rate,AFIB_rate
count,8.100000e+02,810.000000,810.000000,810.000000,810.000000,810.000000,810.000000,810.000000
mean,3.492571e+07,0.230864,0.162963,0.495062,0.416049,0.306173,0.162963,0.312346
std,2.896935e+06,0.421646,0.369560,0.500285,0.493206,0.461187,0.369560,0.463736
min,3.003771e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.236288e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.490289e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.736149e+07,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,3.999923e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [73]:
mido_score_SAPS = score_SAPS_data[score_SAPS_data['stay_id'].isin(cohort_mido['stay_id'].values)]
mido_score_SAPS.describe()

,subject_id,hadm_id,stay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,PaO2FiO2_score,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
count,8.100000e+02,8.100000e+02,8.100000e+02,810.000000,810.000000,810.000000,808.000000,803.000000,803.000000,233.000000,726.000000,803.000000,799.000000,803.000000,803.000000,803.000000,582.000000,806.00000,810.000000,810.000000
mean,1.506504e+07,2.513436e+07,3.492571e+07,44.243210,0.361531,11.035802,1.913366,5.488169,0.149440,7.729614,4.341598,4.134496,0.683354,0.896638,0.348692,1.721046,3.544674,2.07072,0.980247,6.293827
std,2.836112e+06,2.970905e+06,2.896935e+06,14.142096,0.245338,4.930915,1.942780,3.513536,0.653083,1.893689,4.722840,3.592989,1.503917,1.374156,1.083983,1.907301,4.201126,4.82681,3.062934,0.749181
min,1.000892e+07,2.000173e+07,3.003771e+07,11.000000,0.011265,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1.266337e+07,2.243435e+07,3.236288e+07,34.000000,0.152870,7.000000,0.000000,5.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,6.000000
50%,1.496978e+07,2.541225e+07,3.490289e+07,43.000000,0.305597,12.000000,2.000000,5.000000,0.000000,6.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,6.000000
75%,1.746826e+07,2.782586e+07,3.736149e+07,54.000000,0.552904,15.000000,2.000000,5.000000,0.000000,9.000000,11.000000,6.000000,0.000000,3.000000,0.000000,3.000000,9.000000,0.00000,0.000000,6.000000
max,1.999930e+07,2.999661e+07,3.999923e+07,95.000000,0.977893,18.000000,11.000000,13.000000,3.000000,11.000000,11.000000,10.000000,12.000000,3.000000,5.000000,5.000000,9.000000,26.00000,17.000000,8.000000


In [74]:
mido_score_SOFA = score_SOFA_data[score_SOFA_data['stay_id'].isin(cohort_mido['stay_id'].values)]
mido_score_SOFA.describe()

,stay_id,hr,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
count,1.960420e+05,196042.000000,3886.000000,5901.000000,721.000000,61004.000000,1250.000000,771.000000,181886.000000,48625.000000,...,184471.000000,48625.000000,85644.000000,196042.000000,196042.000000,196042.000000,196042.000000,196042.000000,196042.000000,196042.000000
mean,3.496941e+07,278.647254,267.170127,242.588755,0.128860,0.127287,5.656874,5.774538,72.471509,14.416638,...,1.439646,0.324895,0.702875,0.815958,1.002127,1.225860,1.997572,0.640975,1.573979,7.256471
std,2.907848e+06,312.518036,116.661443,135.711129,0.263866,0.125267,4.043923,3.297950,14.006087,1.657111,...,1.511327,0.718408,1.331238,1.161000,1.149343,1.579297,1.412331,1.003156,1.559490,4.393469
min,3.003771e+07,0.000000,30.000000,30.000000,0.004904,0.005000,0.200112,0.500032,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.219791e+07,68.000000,186.000000,156.666667,0.030046,0.049994,2.995316,2.521278,64.000000,15.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.000000
50%,3.493303e+07,176.000000,252.000000,218.000000,0.040047,0.086879,4.875175,5.009404,71.000000,15.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,7.000000
75%,3.729828e+07,375.000000,333.333333,302.857143,0.119430,0.170041,7.988650,7.499096,79.000000,15.000000,...,3.000000,0.000000,0.000000,2.000000,2.000000,3.000000,3.000000,1.000000,3.000000,10.000000
max,3.999923e+07,2390.000000,1313.333333,1990.000000,2.039487,9.503336,22.267373,20.029087,299.000000,15.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,23.000000


In [85]:
mido_SOFA = SOFA_data[SOFA_data['stay_id'].isin(cohort_mido['stay_id'].values)]
mido_SOFA.describe()

,stay_id,sofa_24hours
count,1.960420e+05,196042.000000
mean,3.496941e+07,7.256471
std,2.907848e+06,4.393469
min,3.003771e+07,0.000000
25%,3.219791e+07,4.000000
50%,3.493303e+07,7.000000
75%,3.729828e+07,10.000000
max,3.999923e+07,23.000000


## Divide the Non-Midodrine Group

In [60]:
cohort_nomido = cohort_data[pd.isnull(cohort_data['drug_starttime'])]
cohort_nomido.shape

(34490, 24)

In [61]:
nomido_weights = weight_data[(weight_data['stay_id'].isin(cohort_nomido['stay_id'].values)) & (weight_data['weight_type']=='admit')]
nomido_weights.describe()

,stay_id,weight
count,3.384000e+04,33840.000000
mean,3.498892e+07,80.556265
std,2.893777e+06,26.138179
min,3.000015e+07,1.000000
25%,3.245266e+07,64.700000
50%,3.500354e+07,77.000000
75%,3.749712e+07,92.000000
max,3.999938e+07,1120.000000


In [62]:
nomido_vital_signs = vital_sign_data[vital_sign_data['stay_id'].isin(cohort_nomido['stay_id'].values)]
nomido_vital_signs.describe()

,stay_id,temperature_mean,heart_rate_mean,resp_rate_mean,mbp_mean
count,3.448400e+04,34309.000000,34482.000000,34477.000000,34466.000000
mean,3.498515e+07,36.885414,85.390849,19.321163,80.610642
std,2.892283e+06,0.477306,14.804212,3.649900,11.035871
min,3.000015e+07,30.400000,29.807692,6.000000,24.071429
25%,3.245379e+07,36.658571,74.692308,16.695652,72.892133
50%,3.499529e+07,36.869286,84.751114,18.850000,79.900000
75%,3.749072e+07,37.126111,95.235091,21.474453,87.567889
max,3.999938e+07,40.055000,167.000000,44.333333,139.000000


In [63]:
nomido_labs = lab_tests_data[lab_tests_data['stay_id'].isin(cohort_nomido['stay_id'].values)]
nomido_labs.describe()

,stay_id,hemoglobin,platelet,wbc,sodium,potassium,bicarbonate,chloride,bun,creatinine,lactate,ph,po2,pco2,bnp,troponin,creatinine_kinase
count,3.449000e+04,33362.000000,33363.000000,33364.000000,33467.000000,33471.000000,33455.000000,33465.000000,33474.000000,33483.000000,16943.000000,19566.000000,19567.000000,19567.000000,12265.000000,12265.000000,12265.000000
mean,3.498527e+07,10.716319,214.265971,11.513809,138.811924,4.081628,23.737796,104.216488,23.934935,1.287684,2.134721,7.375918,115.387669,41.702847,0.114486,0.517600,0.799505
std,2.892301e+06,2.003724,107.589393,9.716823,4.573957,0.503174,4.191311,5.639896,19.689209,1.418460,1.884216,0.077543,69.176662,10.168912,0.287098,0.473380,0.356066
min,3.000015e+07,3.633333,6.000000,0.100000,112.100000,1.600000,3.000000,73.266667,1.250000,0.100000,0.050000,6.720000,13.000000,11.000000,0.000000,0.000000,0.000000
25%,3.245407e+07,9.175000,146.250000,7.600000,136.500000,3.755364,21.333333,101.000000,12.000000,0.688889,1.171429,7.336667,67.000000,35.500000,0.000000,0.000000,0.705882
50%,3.499529e+07,10.600000,201.000000,10.200000,139.000000,4.000000,23.800000,104.400000,17.666667,0.900000,1.600000,7.383333,100.000000,40.142857,0.000000,0.666667,1.000000
75%,3.749120e+07,12.133333,264.124521,13.507933,141.264912,4.318750,26.000000,107.500000,28.500000,1.288889,2.400000,7.426667,145.361111,46.000000,0.000000,1.000000,1.000000
max,3.999938e+07,19.757143,2133.000000,471.700000,179.000000,9.600000,49.000000,145.000000,214.375000,32.000000,26.660000,7.690000,587.000000,228.500000,1.000000,1.000000,1.000000


In [64]:
nomido_comorbidities = comorbidities_data[comorbidities_data['stay_id'].isin(cohort_nomido['stay_id'].values)]
nomido_comorbidities.describe()

,stay_id,CPD_or_COPD_rate,Maligancy_rate,mild_liver_rate,severe_liver_rate,Renal_rate,CAD_rate,AFIB_rate
count,3.449000e+04,34490.000000,34490.000000,34490.000000,34490.000000,34490.000000,34490.000000,34490.000000
mean,3.498527e+07,0.233720,0.161264,0.115802,0.047550,0.161264,0.158162,0.216005
std,2.892301e+06,0.423202,0.367780,0.319992,0.212815,0.367780,0.364898,0.411523
min,3.000015e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.245407e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.499529e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.749120e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.999938e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [75]:
nomido_score_SAPS = score_SAPS_data[score_SAPS_data['stay_id'].isin(cohort_nomido['stay_id'].values)]
nomido_score_SAPS.describe()

,subject_id,hadm_id,stay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,PaO2FiO2_score,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
count,3.449000e+04,3.449000e+04,3.449000e+04,34490.000000,34490.000000,34490.000000,34439.000000,34360.000000,34148.000000,6985.000000,33539.000000,33271.000000,33049.000000,33312.000000,33307.000000,33284.000000,15408.000000,34290.000000,34490.000000,34490.000000
mean,1.499812e+07,2.497242e+07,3.498527e+07,33.560771,0.202953,10.979356,1.913528,3.667317,0.132921,7.492054,1.775694,1.820444,0.457593,0.530169,0.200679,0.858581,0.730140,1.993409,1.209423,6.395129
std,2.888636e+06,2.887915e+06,2.892301e+06,14.349046,0.207393,5.868605,1.849093,3.336752,0.617339,1.885763,3.358664,2.969575,1.451850,1.144320,0.717846,1.560942,2.287449,4.837825,3.277214,1.069359
min,1.000003e+07,2.000035e+07,3.000015e+07,0.000000,0.000425,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.249125e+07,2.247460e+07,3.245407e+07,23.000000,0.052195,7.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
50%,1.501106e+07,2.493507e+07,3.499529e+07,32.000000,0.128048,12.000000,2.000000,5.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
75%,1.750577e+07,2.746506e+07,3.749120e+07,42.000000,0.285486,16.000000,2.000000,5.000000,0.000000,9.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
max,1.999999e+07,2.999962e+07,3.999938e+07,114.000000,0.995367,18.000000,11.000000,13.000000,3.000000,11.000000,11.000000,10.000000,12.000000,3.000000,5.000000,5.000000,9.000000,26.000000,17.000000,8.000000


In [76]:
nomido_score_SOFA = score_SOFA_data[score_SOFA_data['stay_id'].isin(cohort_nomido['stay_id'].values)]
nomido_score_SOFA.describe()

,stay_id,hr,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
count,2.768358e+06,2.768358e+06,33901.000000,59914.000000,6372.000000,211564.000000,8914.000000,7449.000000,2.446681e+06,841724.000000,...,2.455916e+06,841724.000000,1.241937e+06,2.768358e+06,2.768358e+06,2.768358e+06,2.768358e+06,2.768358e+06,2.768358e+06,2.768358e+06
mean,3.498344e+07,1.345759e+02,252.548000,241.697397,0.188401,0.149135,7.460420,4.395639,8.078551e+01,14.272860,...,5.180613e-01,0.397249,2.308990e-01,6.381462e-01,4.481949e-01,2.693499e-01,1.012307e+00,6.789563e-01,6.502349e-01,3.697189e+00
std,2.896273e+06,3.328219e+02,117.680322,131.890350,0.338212,0.151233,5.292795,3.117759,1.600865e+01,1.778821,...,1.021865e+00,0.783935,7.971626e-01,1.082866e+00,8.561184e-01,7.787013e-01,1.104881e+00,1.030100e+00,1.140598e+00,3.173686e+00
min,3.000015e+07,0.000000e+00,14.000000,20.000000,0.005000,0.000800,0.499686,0.250048,4.300000e-01,3.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.243942e+07,2.100000e+01,172.000000,148.333333,0.040013,0.050008,3.018336,2.500033,7.000000e+01,14.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,3.499380e+07,5.700000e+01,236.000000,216.000000,0.099973,0.100000,5.594714,3.006880,7.900000e+01,15.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
75%,3.750730e+07,1.520000e+02,316.666667,308.000000,0.181158,0.200402,10.007497,5.033735,9.000000e+01,15.000000,...,1.000000e+00,1.000000,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
max,3.999938e+07,8.903000e+03,1666.666667,2066.666667,4.638808,17.449219,30.320300,32.473437,2.990000e+02,15.000000,...,4.000000e+00,4.000000,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,2.300000e+01


In [86]:
nomido_SOFA = SOFA_data[SOFA_data['stay_id'].isin(cohort_nomido['stay_id'].values)]
nomido_SOFA.describe()

,stay_id,sofa_24hours
count,2.768358e+06,2.768358e+06
mean,3.498344e+07,3.697189e+00
std,2.896273e+06,3.173686e+00
min,3.000015e+07,0.000000e+00
25%,3.243942e+07,1.000000e+00
50%,3.499380e+07,3.000000e+00
75%,3.750730e+07,5.000000e+00
max,3.999938e+07,2.300000e+01
